**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(FD_RES, "source", "ENCODE_E2G", "annotation_e2g_benchmark_ENCSR998YDI")
dir(fdiry)

[1] "distal_regulation_combined_crispr_data_methods-1.pdf"
[2] "K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.tsv"

## Import data

In [5]:
fdiry = file.path(FD_RES, "source", "ENCODE_E2G", "annotation_e2g_benchmark_ENCSR998YDI")
fname = "K562.ENCSR998YDI.ENCFF968BZL.element_gene_links.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, show_col_types = FALSE)

dat_peak_encode_e2g_import = dat
print(dim(dat))
head(dat)

[1] 10412    28


chrom,chromStart,chromEnd,name,EffectSize,strandPerturbationTarget,PerturbationTargetID,chrTSS,startTSS,endTSS,⋯,pValue,pValueAdjusted,PowerAtEffectSize25,PowerAtEffectSize10,PowerAtEffectSize15,PowerAtEffectSize20,PowerAtEffectSize50,ValidConnection,Notes,Reference
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:.,-0.293431866,.,chr1:3691278-3691778:.,chr1,3857213,3857214,⋯,NA,0.004023984,0.8250936,NA,NA,NA,NA,TRUE,Dataset: Fulco2019,"Ulirsch et al., 2016"
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:.,-0.331178093,.,chr1:3691278-3691778:.,chr1,3796503,3796504,⋯,NA,0.007771168,0.6089942,NA,NA,NA,NA,TRUE,Dataset: Fulco2019,"Ulirsch et al., 2016"
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:.,-0.472019217,.,chr1:3691278-3691778:.,chr1,3772761,3772762,⋯,NA,0.000641649,0.6393323,NA,NA,NA,NA,TRUE,Dataset: Fulco2019,"Ulirsch et al., 2016"
chr1,3803570,3805848,LRRC47|chr1:3720134-3722412:.,-0.001471265,.,chr1:3720134-3722412:.,chr1,3796503,3796504,⋯,0.6818652,0.999918431,1.0000000,0.40,0.80,0.85,1,TRUE,Dataset: Gasperini2019,"Gasperini et al., 2019"
chr1,3803570,3805848,SMIM1|chr1:3720134-3722412:.,0.025676924,.,chr1:3720134-3722412:.,chr1,3772788,3772789,⋯,0.3199596,0.994159681,1.0000000,0.80,0.85,1.00,1,TRUE,Dataset: Gasperini2019,"Gasperini et al., 2019"
chr1,4126791,4127291,SMIM1|chr1:4186851-4187351:.,0.023383787,.,chr1:4186851-4187351:.,chr1,3772788,3772789,⋯,0.3058098,0.992197132,0.9500000,0.55,0.80,0.90,1,TRUE,Dataset: Gasperini2019,"Gasperini et al., 2019"


In [4]:
for (txt in colnames(dat)){cat(txt, "\n")}

chrom 
chromStart 
chromEnd 
name 
EffectSize 
strandPerturbationTarget 
PerturbationTargetID 
chrTSS 
startTSS 
endTSS 
strandGene 
EffectSize95ConfidenceIntervalLow 
EffectSize95ConfidenceIntervalHigh 
measuredGeneSymbol 
measuredEnsemblID 
guideSpacerSeq 
guideSeq 
Significant 
pValue 
pValueAdjusted 
PowerAtEffectSize25 
PowerAtEffectSize10 
PowerAtEffectSize15 
PowerAtEffectSize20 
PowerAtEffectSize50 
ValidConnection 
Notes 
Reference 


## Arrange

In [13]:
dat = dat_peak_encode_e2g_import
dat = dat %>% dplyr::select(Significant, pValue, pValueAdjusted, ValidConnection)
head(dat)

Significant,pValue,pValueAdjusted,ValidConnection
<lgl>,<dbl>,<dbl>,<lgl>
TRUE,NA,0.004023984,TRUE
TRUE,NA,0.007771168,TRUE
TRUE,NA,0.000641649,TRUE
FALSE,0.6818652,0.999918431,TRUE
FALSE,0.3199596,0.994159681,TRUE
FALSE,0.3058098,0.992197132,TRUE


In [12]:
table(dat$Significant)


FALSE  TRUE 
 9843   569 

In [8]:
dat = dat_peak_encode_e2g_import
dat = dat %>% 
    dplyr::rename(
        Chrom = chrom, 
        Start = chromStart, 
        End   = chromEnd, 
        Name  = name)

txt = "ENCODE-E2G_Benchmark"
dat = dat %>% 
    tidyr::separate(Name, c("Gene_Symbol", "Loc"), sep="\\|", remove = FALSE) %>% 
    dplyr::mutate(
        EffectSize   = round(EffectSize, 5),
        Region       = paste0(Chrom,  ":", Start,    "-", End),
        Gene_TSS     = paste0(chrTSS, ":", startTSS, "-", endTSS),
        Assay_Type   = txt,
        Assay_Label  = paste0(txt, ":", Gene_Symbol)
    )

vec = c('Chrom','Start','End','Name','EffectSize', 
        "Region", "Gene_Symbol", "Gene_TSS", 
        'Significant', 
        'Reference', 
        'Assay_Type', 'Assay_Label')
dat = dat %>% 
    dplyr::select({{vec}}) %>% 
    dplyr::distinct()
vec = dat$Gene_Symbol


dat_peak_encode_e2g_arrange = dat
cat("#Row: ", nrow(dat), "\n")
cat("#Gene:", length(unique(vec)), "\n")
head(dat)

#Row:  10412 
#Gene: 2146 


Chrom,Start,End,Name,EffectSize,Region,Gene_Symbol,Gene_TSS,Significant,Reference,Assay_Type,Assay_Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:.,-0.29343,chr1:3774714-3775214,CEP104,chr1:3857213-3857214,TRUE,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:CEP104
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:.,-0.33118,chr1:3774714-3775214,LRRC47,chr1:3796503-3796504,TRUE,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:LRRC47
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:.,-0.47202,chr1:3774714-3775214,SMIM1,chr1:3772761-3772762,TRUE,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1
chr1,3803570,3805848,LRRC47|chr1:3720134-3722412:.,-0.00147,chr1:3803570-3805848,LRRC47,chr1:3796503-3796504,FALSE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:LRRC47
chr1,3803570,3805848,SMIM1|chr1:3720134-3722412:.,0.02568,chr1:3803570-3805848,SMIM1,chr1:3772788-3772789,FALSE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1
chr1,4126791,4127291,SMIM1|chr1:4186851-4187351:.,0.02338,chr1:4126791-4127291,SMIM1,chr1:3772788-3772789,FALSE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1


## Define active and inactive regions

In [9]:
dat = dat_peak_encode_e2g_arrange
dat = dat %>% 
    dplyr::filter(Significant) %>%
    dplyr::arrange(Chrom, Start, End)

dat_peak_encode_e2g_active = dat
print(dim(dat))
head(dat)

[1] 569  12


Chrom,Start,End,Name,EffectSize,Region,Gene_Symbol,Gene_TSS,Significant,Reference,Assay_Type,Assay_Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:.,-0.29343,chr1:3774714-3775214,CEP104,chr1:3857213-3857214,TRUE,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:CEP104
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:.,-0.33118,chr1:3774714-3775214,LRRC47,chr1:3796503-3796504,TRUE,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:LRRC47
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:.,-0.47202,chr1:3774714-3775214,SMIM1,chr1:3772761-3772762,TRUE,"Ulirsch et al., 2016",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1
chr1,8858063,8858563,ENO1|chr1:8918122-8918622:.,-0.10741,chr1:8858063-8858563,ENO1,chr1:8879248-8879249,TRUE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:ENO1
chr1,17441310,17441810,SDHB|chr1:17767806-17768306:.,-0.08968,chr1:17441310-17441810,SDHB,chr1:17054169-17054170,TRUE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SDHB
chr1,17487740,17488240,RP4-798A10.7|chr1:17814236-17814736:.,0.03966,chr1:17487740-17488240,RP4-798A10.7,chr1:16514645-16514646,TRUE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:RP4-798A10.7


In [6]:
dat = dat_peak_encode_e2g_active
dat = dat %>%
    dplyr::select(Chrom, Start, End) %>%
    dplyr::distinct()

print(dim(dat))

[1] 429   3


In [7]:
vec_txt_region_total    = unique(dat_peak_encode_e2g_arrange$Region)
vec_txt_region_active   = unique(dat_peak_encode_e2g_active$Region)
vec_txt_region_inactive = setdiff(vec_txt_region_total, vec_txt_region_active)

dat = dat_peak_encode_e2g_arrange
dat = dat %>% 
    dplyr::filter(!Regulated) %>%
    dplyr::filter(Region %in% vec_txt_region_inactive) %>%
    dplyr::select(Chrom, Start, End) %>%
    dplyr::distinct() %>%
    dplyr::arrange(Chrom, Start, End)

dat_peak_encode_e2g_inactive = dat
print(dim(dat))
head(dat)

[1] 3531    3


Chrom,Start,End
<chr>,<dbl>,<dbl>
chr1,3803570,3805848
chr1,4126791,4127291
chr1,5304578,5305078
chr1,8197448,8198244
chr1,8899850,8900350
chr1,9145155,9145655


In [8]:
dat = dat_peak_encode_e2g_arrange
dat = dat %>% 
    dplyr::select(Chrom, Start, End) %>%
    dplyr::distinct() %>%
    dplyr::arrange(Chrom, Start, End)

dat_peak_encode_e2g_total = dat
print(dim(dat))
head(dat)

[1] 3961    3


Chrom,Start,End
<chr>,<dbl>,<dbl>
chr1,3774714,3775214
chr1,3803570,3805848
chr1,4126791,4127291
chr1,5304578,5305078
chr1,8197448,8198244
chr1,8858063,8858563


## Data dictionary

In [9]:
dat_cnames = tribble(
    ~Name,         ~Description,
    "Chrom",       "Chromosome of the perturbed element",
    "Start",       "Start of the perturbed element (0-based)",
    "End",         "End of the perturbed element",
    "Name",        "Identifier for the given E-G pair in the form of 'gene|chr:start-stop:strand', e.g. 'BET1L|chr11:220536-220704:*'",
    "Score",       "Change in gene expression upon CRISPRi perturbation, e.g. precentage change in expression or log fold change",
    "Region",      "Location of the perturbed element",
    "Gene_Symbol", "target gene symbol",
    "Gene_TSS",    "target gene TSS",
    "Reference",   "Study from which this E-G pair was collected, e.g. Fulco2019",
    "Assay_Type",  "Assay type",
    "Assay_Label", "Assay label"
)
dat_cnames

Name,Description
<chr>,<chr>
Chrom,Chromosome of the perturbed element
Start,Start of the perturbed element (0-based)
End,End of the perturbed element
Name,"Identifier for the given E-G pair in the form of 'gene|chr:start-stop:strand', e.g. 'BET1L|chr11:220536-220704:*'"
Score,"Change in gene expression upon CRISPRi perturbation, e.g. precentage change in expression or log fold change"
Region,Location of the perturbed element
Gene_Symbol,target gene symbol
Gene_TSS,target gene TSS
Reference,"Study from which this E-G pair was collected, e.g. Fulco2019"


## Interact for active regions

In [10]:
txt = c('chrom',  'chromStart', 'chromEnd',
        "chrTSS", "startTSS",   "endTSS",
        "name", 
        "EffectSize", 
        "Strand1", "Strand2",
        "Gene_Symbol", "Gene_TSS", "Reference")


dat = dat_peak_encode_e2g_import
dat = dat %>% 
    dplyr::filter(Regulated) %>%
    tidyr::separate(name, c("Gene_Symbol", "Loc"), sep="\\|", remove = FALSE) %>% 
    dplyr::mutate(
        EffectSize   = round(EffectSize, 5),
        Gene_TSS     = paste0(chrTSS, ":", startTSS, "-", endTSS),
        Strand1      = ".",
        Strand2      = "."
    )

dat = dat %>% 
    dplyr::select({{txt}}) %>% 
    dplyr::arrange(chrom, chromStart, chromEnd)

dat_peak_encode_e2g_bedpe = dat
print(dim(dat))
head(dat)

[1] 487  13


chrom,chromStart,chromEnd,chrTSS,startTSS,endTSS,name,EffectSize,Strand1,Strand2,Gene_Symbol,Gene_TSS,Reference
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,3774714,3775214,chr1,3857213,3857214,CEP104|chr1:3691278-3691778:*,-0.29343,.,.,CEP104,chr1:3857213-3857214,Ulirsch2016
chr1,3774714,3775214,chr1,3796503,3796504,LRRC47|chr1:3691278-3691778:*,-0.33118,.,.,LRRC47,chr1:3796503-3796504,Ulirsch2016
chr1,3774714,3775214,chr1,3772761,3772762,SMIM1|chr1:3691278-3691778:*,-0.47202,.,.,SMIM1,chr1:3772761-3772762,Ulirsch2016
chr1,8858063,8858563,chr1,8879248,8879249,ENO1|chr1:8918122-8918622:.,-0.10741,.,.,ENO1,chr1:8879248-8879249,"Gasperini et al., 2019"
chr1,17441310,17441810,chr1,17054169,17054170,SDHB|chr1:17767806-17768306:.,-0.08968,.,.,SDHB,chr1:17054169-17054170,"Gasperini et al., 2019"
chr1,25933351,25933954,chr1,26900238,26900239,NUDC|chr1:26259842-26260445:.,-0.08462,.,.,NUDC,chr1:26900238-26900239,"Gasperini et al., 2019"


In [11]:
dat = dat_peak_encode_e2g_bedpe

dat = dat %>%
    dplyr::mutate(score = 1000, value = 1, exp = ".", color = ".") %>%
    dplyr::mutate(
        chromMid = as.integer((chromStart + chromEnd) / 2),
        sourceName = paste0(chrom, ":", chromStart, "-", chromEnd),
        targetName = Gene_TSS,
    ) %>%
    dplyr::mutate(
        sourceChrom   = chrom,
        sourceStart   = chromStart,
        sourceEnd     = chromEnd,
        sourceStrand  = ".",
        targetChrom   = chrTSS,
        targetStart   = startTSS,
        targetEnd     = endTSS,
        targetStrand  = "."
    ) %>%
    dplyr::select(
        chrom, chromStart, chromEnd, name, 
        score, value, exp, color,
        sourceChrom,
        sourceStart,
        sourceEnd,
        sourceName,
        sourceStrand,
        targetChrom,
        targetStart,
        targetEnd,
        targetName,
        targetStrand
    )

dat_peak_encode_e2g_interact = dat
print(dim(dat))
head(dat)

[1] 487  18


chrom,chromStart,chromEnd,name,score,value,exp,color,sourceChrom,sourceStart,sourceEnd,sourceName,sourceStrand,targetChrom,targetStart,targetEnd,targetName,targetStrand
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:*,1000,1,.,.,chr1,3774714,3775214,chr1:3774714-3775214,.,chr1,3857213,3857214,chr1:3857213-3857214,.
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:*,1000,1,.,.,chr1,3774714,3775214,chr1:3774714-3775214,.,chr1,3796503,3796504,chr1:3796503-3796504,.
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:*,1000,1,.,.,chr1,3774714,3775214,chr1:3774714-3775214,.,chr1,3772761,3772762,chr1:3772761-3772762,.
chr1,8858063,8858563,ENO1|chr1:8918122-8918622:.,1000,1,.,.,chr1,8858063,8858563,chr1:8858063-8858563,.,chr1,8879248,8879249,chr1:8879248-8879249,.
chr1,17441310,17441810,SDHB|chr1:17767806-17768306:.,1000,1,.,.,chr1,17441310,17441810,chr1:17441310-17441810,.,chr1,17054169,17054170,chr1:17054169-17054170,.
chr1,25933351,25933954,NUDC|chr1:26259842-26260445:.,1000,1,.,.,chr1,25933351,25933954,chr1:25933351-25933954,.,chr1,26900238,26900239,chr1:26900238-26900239,.


## Save results

In [12]:
FD_OUT = file.path(FD_RES, "results", "region", "annotation_encode_e2g")

In [13]:
fdiry = FD_OUT
fname = "description.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_cnames
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g/description.tsv"


In [14]:
fdiry = FD_OUT
fname = "K562.EPCrisprBenchmark.total.bed.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_total
colnames(dat)[1] = "#Chrom"
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g/K562.EPCrisprBenchmark.total.bed.gz"


In [15]:
fdiry = FD_OUT
fname = "K562.EPCrisprBenchmark.active.bed.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_active
colnames(dat)[1] = "#Chrom"
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g/K562.EPCrisprBenchmark.active.bed.gz"


In [16]:
fdiry = FD_OUT
fname = "K562.EPCrisprBenchmark.inactive.bed.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_inactive
colnames(dat)[1] = "#Chrom"
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g/K562.EPCrisprBenchmark.inactive.bed.gz"


In [17]:
fdiry = FD_OUT
fname = "K562.EPCrisprBenchmark.active.region2TSS.bedpe.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_bedpe
colnames(dat)[1] = "#chrom"
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g/K562.EPCrisprBenchmark.active.region2TSS.bedpe.gz"


In [18]:
fdiry = FD_OUT
fname = "K562.EPCrisprBenchmark.active.region2TSS.bed"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_encode_e2g_interact
colnames(dat)[1] = "#chrom"
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_encode_e2g/K562.EPCrisprBenchmark.active.region2TSS.bed"


In [8]:
txt = c('chrom','chromStart','chromEnd','name','EffectSize', 
        "Gene_Symbol", "Gene_TSS", 
        'Reference', 
        'Assay_Type', 'Assay_Label')
vec = c(
    "Gasperini et al., 2019", 
    "Fulco2019", 
    "Schraivogel et al., 2020", 
    "Klann2017", 
    "Fulco2016", 
    "Thakore2015")

dat = dat_peak_crispri_e2g_import
dat = dat %>% 
    dplyr::filter(Regulated) %>% 
    dplyr::filter(Reference %in% vec)

dat = dat %>% 
    tidyr::separate(name, c("Gene_Symbol", "Loc"), sep="\\|", remove = FALSE) %>% 
    dplyr::mutate(
        EffectSize   = round(EffectSize, 5),
        Gene_TSS     = paste0(chrTSS, ":", startTSS, "-", endTSS),
        Assay_Type   = "CRISPRi-E2G",
        Assay_Label  = paste0("CRISPRi-E2G", ":", Gene_Symbol)
    )
        #Assay_Label2 = paste0("CRISPRi-E2G", "(", Reference, ")", ":", Gene_Symbol)

dat = dat %>% 
    dplyr::select({{txt}}) %>% 
    dplyr::arrange(chrom, chromStart, chromEnd)

dat_peak_crispri_e2g_arrange = dat
print(dim(dat))
head(dat)

[1] 464  10


chrom,chromStart,chromEnd,name,EffectSize,Gene_Symbol,Gene_TSS,Reference,Assay_Type,Assay_Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,8858063,8858563,ENO1|chr1:8918122-8918622:.,-0.10741,ENO1,chr1:8879248-8879249,"Gasperini et al., 2019",CRISPRi-E2G,CRISPRi-E2G:ENO1
chr1,17441310,17441810,SDHB|chr1:17767806-17768306:.,-0.08968,SDHB,chr1:17054169-17054170,"Gasperini et al., 2019",CRISPRi-E2G,CRISPRi-E2G:SDHB
chr1,25933351,25933954,NUDC|chr1:26259842-26260445:.,-0.08462,NUDC,chr1:26900238-26900239,"Gasperini et al., 2019",CRISPRi-E2G,CRISPRi-E2G:NUDC
chr1,25933351,25933954,STMN1|chr1:26259842-26260445:.,-0.12223,STMN1,chr1:25906990-25906991,"Gasperini et al., 2019",CRISPRi-E2G,CRISPRi-E2G:STMN1
chr1,26377523,26378866,LIN28A|chr1:26704014-26705357:.,-0.05172,LIN28A,chr1:26410778-26410779,"Gasperini et al., 2019",CRISPRi-E2G,CRISPRi-E2G:LIN28A
chr1,28883765,28884265,EPB41|chr1:29210277-29210777:.,-0.26521,EPB41,chr1:28887091-28887092,"Gasperini et al., 2019",CRISPRi-E2G,CRISPRi-E2G:EPB41


In [9]:
txt = c('chrom',  'chromStart', 'chromEnd',
        "chrTSS", "startTSS",   "endTSS",
        'name', 'EffectSize', 
        "Strand1", "Strand2",
        "Gene_Symbol", "Gene_TSS", 
        'Reference')

#txt = c('chrom',  'chromStart', 'chromEnd',
#        "chrTSS", "startTSS",   "endTSS",
#        'name')

vec = c(
    "Gasperini et al., 2019", 
    "Fulco2019", 
    "Schraivogel et al., 2020", 
    "Klann2017", 
    "Fulco2016", 
    "Thakore2015")

dat = dat_peak_crispri_e2g_import
dat = dat %>% 
    dplyr::filter(Regulated) %>% 
    dplyr::filter(Reference %in% vec)

dat = dat %>% 
    tidyr::separate(name, c("Gene_Symbol", "Loc"), sep="\\|", remove = FALSE) %>% 
    dplyr::mutate(
        EffectSize   = round(EffectSize, 5),
        Gene_TSS     = paste0(chrTSS, ":", startTSS, "-", endTSS),
        Strand1      = ".",
        Strand2      = "."
    )
        #Assay_Label2 = paste0("CRISPRi-E2G", "(", Reference, ")", ":", Gene_Symbol)

dat = dat %>% 
    dplyr::select({{txt}}) %>% 
    dplyr::arrange(chrom, chromStart, chromEnd)

dat_peak_crispri_e2g_bedpe = dat
print(dim(dat))
head(dat)

[1] 464  13


chrom,chromStart,chromEnd,chrTSS,startTSS,endTSS,name,EffectSize,Strand1,Strand2,Gene_Symbol,Gene_TSS,Reference
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
chr1,8858063,8858563,chr1,8879248,8879249,ENO1|chr1:8918122-8918622:.,-0.10741,.,.,ENO1,chr1:8879248-8879249,"Gasperini et al., 2019"
chr1,17441310,17441810,chr1,17054169,17054170,SDHB|chr1:17767806-17768306:.,-0.08968,.,.,SDHB,chr1:17054169-17054170,"Gasperini et al., 2019"
chr1,25933351,25933954,chr1,26900238,26900239,NUDC|chr1:26259842-26260445:.,-0.08462,.,.,NUDC,chr1:26900238-26900239,"Gasperini et al., 2019"
chr1,25933351,25933954,chr1,25906990,25906991,STMN1|chr1:26259842-26260445:.,-0.12223,.,.,STMN1,chr1:25906990-25906991,"Gasperini et al., 2019"
chr1,26377523,26378866,chr1,26410778,26410779,LIN28A|chr1:26704014-26705357:.,-0.05172,.,.,LIN28A,chr1:26410778-26410779,"Gasperini et al., 2019"
chr1,28883765,28884265,chr1,28887091,28887092,EPB41|chr1:29210277-29210777:.,-0.26521,.,.,EPB41,chr1:28887091-28887092,"Gasperini et al., 2019"


In [10]:
dat = dat_peak_crispri_e2g_bedpe

dat = dat %>%
    dplyr::mutate(score = 1000, value = 1, exp = ".", color = ".") %>%
    dplyr::mutate(
        chromMid = as.integer((chromStart + chromEnd) / 2),
        sourceName = paste0(chrom, ":", chromStart, "-", chromEnd),
        targetName = Gene_TSS,
    ) %>%
    dplyr::mutate(
        #name          = ".",
        sourceChrom   = chrom,
        sourceStart   = chromStart,
        sourceEnd     = chromEnd,
        #sourceName    = ".",
        sourceStrand  = ".",
        targetChrom   = chrTSS,
        targetStart   = startTSS,
        targetEnd     = endTSS,
        #targetName    = ".",
        targetStrand  = "."
    ) %>%
    dplyr::select(
        chrom, chromStart, chromEnd, name, 
        score, value, exp, color,
        sourceChrom,
        sourceStart,
        sourceEnd,
        sourceName,
        sourceStrand,
        targetChrom,
        targetStart,
        targetEnd,
        targetName,
        targetStrand
    )

dat_peak_crispri_e2g_interact = dat
print(dim(dat))
head(dat)

[1] 464  18


chrom,chromStart,chromEnd,name,score,value,exp,color,sourceChrom,sourceStart,sourceEnd,sourceName,sourceStrand,targetChrom,targetStart,targetEnd,targetName,targetStrand
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
chr1,8858063,8858563,ENO1|chr1:8918122-8918622:.,1000,1,.,.,chr1,8858063,8858563,chr1:8858063-8858563,.,chr1,8879248,8879249,chr1:8879248-8879249,.
chr1,17441310,17441810,SDHB|chr1:17767806-17768306:.,1000,1,.,.,chr1,17441310,17441810,chr1:17441310-17441810,.,chr1,17054169,17054170,chr1:17054169-17054170,.
chr1,25933351,25933954,NUDC|chr1:26259842-26260445:.,1000,1,.,.,chr1,25933351,25933954,chr1:25933351-25933954,.,chr1,26900238,26900239,chr1:26900238-26900239,.
chr1,25933351,25933954,STMN1|chr1:26259842-26260445:.,1000,1,.,.,chr1,25933351,25933954,chr1:25933351-25933954,.,chr1,25906990,25906991,chr1:25906990-25906991,.
chr1,26377523,26378866,LIN28A|chr1:26704014-26705357:.,1000,1,.,.,chr1,26377523,26378866,chr1:26377523-26378866,.,chr1,26410778,26410779,chr1:26410778-26410779,.
chr1,28883765,28884265,EPB41|chr1:29210277-29210777:.,1000,1,.,.,chr1,28883765,28884265,chr1:28883765-28884265,.,chr1,28887091,28887092,chr1:28887091-28887092,.


In [12]:
dat = dat_peak_crispri_e2g_arrange
print(dim(dat))
table(dat$Assay_Label)

[1] 464  10



        CRISPRi-E2G:ABCD1    CRISPRi-E2G:AC002480.3    CRISPRi-E2G:AC073072.5 
                        1                         2                         1 
     CRISPRi-E2G:ADAMTS14         CRISPRi-E2G:AGAP1         CRISPRi-E2G:AGFG1 
                        1                         1                         1 
          CRISPRi-E2G:AK1        CRISPRi-E2G:AKAP11        CRISPRi-E2G:AKR1C1 
                        1                         2                         1 
       CRISPRi-E2G:ANP32B        CRISPRi-E2G:ANTXR2         CRISPRi-E2G:ANXA1 
                        1                         1                         2 
         CRISPRi-E2G:APIP         CRISPRi-E2G:APLP2         CRISPRi-E2G:ARL4A 
                        1                         1                         1 
        CRISPRi-E2G:ASAH1          CRISPRi-E2G:ASB1         CRISPRi-E2G:ASH2L 
                        1                         1                         1 
       CRISPRi-E2G:ATP1B3        CRISPRi-E2G:ATP5G1

In [13]:
dat_cnames = tribble(
    ~Name,      ~Description,
    "Chrom",    "Chromosome of the perturbed element",
    "Start",    "Start of the perturbed element (0-based)",
    "End",      "End of the perturbed element",
    "Name",     "Identifier for the given E-G pair in the form of 'gene|chr:start-stop:strand', e.g. 'BET1L|chr11:220536-220704:*'",
    "Score",     "Change in gene expression upon CRISPRi perturbation, e.g. precentage change in expression or log fold change",
    "Gene_Symbol", "target gene symbol",
    "Gene_TSS",    "target gene TSS",
    "Reference", "Study from which this E-G pair was collected, e.g. Fulco2019",
    "Assay_Type",  "Assay type",
    "Assay_Label", "Assay label"
)
dat_cnames

Name,Description
<chr>,<chr>
Chrom,Chromosome of the perturbed element
Start,Start of the perturbed element (0-based)
End,End of the perturbed element
Name,"Identifier for the given E-G pair in the form of 'gene|chr:start-stop:strand', e.g. 'BET1L|chr11:220536-220704:*'"
Score,"Change in gene expression upon CRISPRi perturbation, e.g. precentage change in expression or log fold change"
Gene_Symbol,target gene symbol
Gene_TSS,target gene TSS
Reference,"Study from which this E-G pair was collected, e.g. Fulco2019"
Assay_Type,Assay type


## Save results

In [11]:
fdiry = file.path(FD_RES, "results", "region", "annotation_crispri_e2g")
fname = "description.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_cnames
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_crispri_e2g/description.tsv"


In [12]:
fdiry = file.path(FD_RES, "results", "region", "annotation_crispri_e2g")
fname = "crispri_e2g.region.bed.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_crispri_e2g_arrange
write_tsv(dat, fpath, col_names = FALSE)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_crispri_e2g/crispri_e2g.region.bed.gz"


In [13]:
fdiry = file.path(FD_RES, "results", "region", "annotation_crispri_e2g")
fname = "crispri_e2g.region2TSS.bedpe.gz"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_crispri_e2g_bedpe
colnames(dat)[1] = "#chrom"
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_crispri_e2g/crispri_e2g.region2TSS.bedpe.gz"


In [19]:
fdiry = file.path(FD_RES, "results", "region", "annotation_crispri_e2g")
fname = "crispri_e2g.region2TSS.inter.bed"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_crispri_e2g_interact
colnames(dat)[1] = "#chrom"
write_tsv(dat, fpath, col_names = FALSE)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_crispri_e2g/crispri_e2g.region2TSS.inter.bed"
